**I think we can start here with preprocessing and building up a pipeline for classifying with a traditional model from pose estimation**

# Claases

- Walking
- Standing
- Fast walking / jogging?
- Running

In [ ]:
# import kagglehub
# Download latest version from kagglehub: 

# path = kagglehub.dataset_download("sharjeelmazhar/human-activity-recognition-video-dataset")

path = r"C:\Users\marte\.cache\kagglehub\datasets\sharjeelmazhar\human-activity-recognition-video-dataset\versions\1"

print("Path to dataset files:", path)

Path to dataset files: C:\Users\marte\.cache\kagglehub\datasets\sharjeelmazhar\human-activity-recognition-video-dataset\versions\1


In [2]:
import os

walking_class = "Human Activity Recognition - Video Dataset/Walking"
standing_class = "Human Activity Recognition - Video Dataset/Standing Still"
walking_path = os.path.join(path, walking_class)
standing_path = os.path.join(path, standing_class)

videos_walking = os.listdir(walking_path)
print(f"Found {len(videos_walking)} videos in class {walking_class}")
print(videos_walking[:5])

videos_standing = os.listdir(standing_path)
print(f"Found {len(videos_standing)} videos in class {standing_class}")
print(videos_standing[:5])

Found 171 videos in class Human Activity Recognition - Video Dataset/Walking
['Walking (1).mp4', 'Walking (10).mp4', 'Walking (100).mp4', 'Walking (101).mp4', 'Walking (102).mp4']
Found 174 videos in class Human Activity Recognition - Video Dataset/Standing Still
['Standing Still (1).mp4', 'Standing Still (10).mp4', 'Standing Still (100).mp4', 'Standing Still (101).mp4', 'Standing Still (102).mp4']


In [3]:
import cv2
import os
import mediapipe as mp

# Video file
video_file = os.path.join(walking_path, videos_walking[0])
cap = cv2.VideoCapture(video_file)

if not cap.isOpened():
    print("Error opening video:", video_file)
    
else:
    # Create a resizable window
    cv2.namedWindow('Video', cv2.WINDOW_NORMAL)
    #cv2.resizeWindow('Video', 1280, 720)

    # Initialize MediaPipe Pose
    mp_pose = mp.solutions.pose
    mp_drawing = mp.solutions.drawing_utils

    with mp_pose.Pose() as pose:

        while True:
            ret, frame = cap.read()
            if not ret:
                break  # video ended

            # Process frame with MediaPipe Pose
            results = pose.process(frame)

            # Draw skeleton on the frame
            if results.pose_landmarks:
                mp_drawing.draw_landmarks(
                    frame,
                    results.pose_landmarks, # The point coordinates on landmarks (mostly joints)
                    mp_pose.POSE_CONNECTIONS, # Draws a line between each landmark, connecting them like a graph with nodes and edges
                )

            # Show the frame
            cv2.imshow('Video', frame)

            if cv2.waitKey(2) & 0xFF == ord('q'):
                break

cap.release()
cv2.destroyAllWindows()